In [1]:
# For development purposes, reload imported modules when source changes
%load_ext autoreload
%autoreload 2

import spikeinterface as si
import spikewidgets as sw
import spiketoolkit as st
import os, sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Create a toy example dataset
recording,sorting_true=sw.example_datasets.toy_example1(duration=60)


# MEArec
#rec_folder = '/home/alessio/Documents/Codes/MEArec/data/recordings/recordings_20cells_Neuronexus-32_10.0_10.0uV_15-10-2018:21:52'
#recording = si.MEArecRecordingExtractor(rec_folder)
#sorting_true = si.MEArecSortingExtractor(rec_folder)

In [ ]:
# View the filtered timeseries data
filt=st.filters.bandpass_filter(recording,freq_min=300,freq_max=6000)
sw.TimeseriesWidget(recording=filt,width=7,height=2.5).display()

In [4]:
# Create a temporary working directory if needed
if not os.path.exists('tmp'): os.mkdir('tmp')

In [10]:
pwd

'/home/alessio/Documents/Codes/spike_sorting/spiketoolkit/examples'

In [ ]:
# Mountainsort4 spike sorting
sorting_MS4=st.sorters.mountainsort4(
    recording=recording,
    detect_sign=-1,
    adjacency_radius=-1
)

In [ ]:
# SpyKING Circus spike sorting
sorting_SC=st.sorters.spyking_circus(
    recording=recording,
    adjacency_radius=50,
    detect_sign='negative',
    output_folder='tmp'
)

In [ ]:
# KiloSort spike sorting
sorting_KS=st.sorters.kilosort_j(
    recording=recording,
    kilosort_src='/home/alessio/Documents/Codes/spike_sorting/KiloSort',
    ironclust_src='/home/alessio/Documents/Codes/spike_sorting/ironclust',
    tmpdir='tmp'
)

In [15]:
sorting_KS=st.sorters.kilosort(
    recording=recording,
    kilosort_path='/home/alessio/Documents/Codes/spike_sorting/KiloSort',
    npy_matlab_path='/home/alessio/Documents/Codes/npy-matlab',
    output_folder='tmp'
)

Running KiloSort
matlab -nosplash -nodisplay -r "run /home/alessio/Documents/Codes/spike_sorting/spiketoolkit/examples/tmp/kilosort/kilosort_master.m; quit;"


Index exceeds matrix dimensions.

                            < M A T L A B (R) >



                  Copyright 1984-2017 The MathWorks, Inc.

Error in fullMPMU (line 205)

                   R2017a (9.2.0.556344) 64-bit (glnxa64)

[~, isort]      = sort(st3(:,1), 'ascend');

                               March 27, 2017





Error in kilosort_master (line 22)

 

rez                = fullMPMU(rez, DATA);% extract final spike times

To get started, type one of these: helpwin, helpdesk, or demo.

(overlapping extraction)

For product information, visit www.mathworks.com.



 

Error in run (line 86)

Time   0s. Loading raw data... 

evalin('caller', [script ';']);

Time   1s. Channel-whitening filters computed. 

 

Time   1s. Loading raw data and applying filters... 

Time 1.35. Whitened data written to disk... 

Time 1.35. Pre

FileNotFoundError: [Errno 2] No such file or directory: '/home/alessio/Documents/Codes/spike_sorting/spiketoolkit/examples/tmp/kilosort/results/spike_times.npy'

In [ ]:
sorting=st.sorters.klusta(
        recording, # The recording extractor
        output_folder='tmp',
        probe_file=None,
        file_name=None,
        threshold_strong_std_factor=5,
        threshold_weak_std_factor=2,
        detect_spikes='negative',
        extract_s_before=16,
        extract_s_after=32,
        n_features_per_channel=3,
        pca_n_waveforms_max=10000,
        num_starting_clusters=50
        )

In [ ]:
# IronClust spike sorting
sorting_IC=st.sorters.ironclust(
    recording=recording,
    detect_sign=-1,
    adjacency_radius=-1,
    prm_template_name='tetrode_template.prm',
    tmpdir='tmp',
    ironclust_src='../ironclust'
)

In [ ]:
# Compute SNRs of true units
snrs=sw.devel.compute_unit_snrs(recording=filt,sorting=sorting_true,unit_ids=sorting_true.getUnitIds())
sorting_true.setUnitsProperty(property_name='SNR',values=snrs)

In [ ]:
# Comparison MountainSort4 vs truth
comparison_MS4=sw.SortingComparison(sorting_true,sorting_MS4)
sw.SortingAccuracyWidget(sorting_comparison=comparison_MS4,property_name='SNR').plot()

In [ ]:
# Comparison SpyKING Circus vs truth
comparison_SC=sw.SortingComparison(sorting_true,sorting_SC)
sw.SortingAccuracyWidget(sorting_comparison=comparison_SC,property_name='SNR').plot()

In [ ]:
# Comparison IronClust vs truth
comparison_IC=sw.SortingComparison(sorting_true,sorting_IC)
sw.SortingAccuracyWidget(sorting_comparison=comparison_IC,property_name='SNR').plot()